In [ ]:
pwd

In [ ]:
cd ..

In [ ]:
from constants import GOV_API_KEY, PST_PHONE_NUMBER, IPHONE_PHONE_NUMBER

In [ ]:
from flask_app.app import score

from notifications_python_client.notifications import NotificationsAPIClient

notifications_client = NotificationsAPIClient(GOV_API_KEY)

In [ ]:
import flask
from flask import Flask, request
import json
import pandas as pd
import pickle
import xgboost
from flask_app.extraction import url_extractor

import regex as re

import urllib.parse as urlparse
from tld import get_tld
import urllib.request as urlreq

In [ ]:
### Greeting msg / autoreply msg
greeting_msg_template_id = 'fbd7abfa-1bf3-4fc2-9948-4a37a6ce9293'

#### Template Spam Msgs
spam_msg_tempalte_id = "ed5edbf5-76e0-42be-9c06-3b3717ce00ad"
not_spam_msg_template_id = "5e107ed8-c631-492f-ad7d-91c73919a42e"

In [ ]:
### Send text message:
phone_number = PST_PHONE_NUMBER

response = notifications_client.send_sms_notification(
    phone_number=phone_number, # required string
    template_id=greeting_msg_template_id, # required UUID string
)

In [ ]:
response

In [ ]:
def get_received_texts():
    """Get all the recieved text messages to the digital phone number"""
    notifications_client.get_recieved_texts()

In [ ]:
def get_spam_score(url):
    
    """Get the spam score of a given url"""
    
    # Convert to features df
    data = url_extractor(url)
    
    # Load model
    model = pickle.load(open('xg_model.pkl', 'rb'))
    
    spam_score = round(model.predict_proba(data.to_numpy())[0][1], 2)

    return spam_score

In [ ]:
def get_spam_classification(spam_score:float, threshold:float=0.65):
    """Get the text classfication or a spam score"""
    if spam_score >= threshold:
        spam_classfication = "SPAM"
    else:
        spam_classfication = "NOT SPAM"
    
    return spam_classfication

In [ ]:
google_url = "https://google.com"

google_spam_score = get_spam_score(google_url)

google_spam_score

In [ ]:
get_spam_classification(spam_score=google_spam_score)

In [ ]:
spam_url = "https://evri.id-60.com"

spam_eg_spam_score = get_spam_score(spam_url)

spam_eg_spam_score

In [ ]:
get_spam_classification(spam_score=spam_eg_spam_score)